## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: **Matias**
- Apellido: **Intraguglielmo**
- Legajo: **164.683-7**

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
# Importo el dataset para luego analizarlo
london = pd.read_csv('TP2_airbnb_london.csv')
# Verifico que se haya cargado correctamente.
london.head(10)

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285
5,7,Islington,Entire home/apt,150,3,54,0.57,7,350
6,8,Islington,Entire home/apt,150,2,67,0.71,7,353
7,9,Tower Hamlets,Private room,50,3,38,0.51,3,69
8,10,Barnet,Entire home/apt,185,7,1,0.02,2,0
9,11,Southwark,Private room,64,3,28,1.01,2,0


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [4]:
# Con .shape podemos ver el tamaño del dataframe por filas y columnas
print(f"Las dimensiones del dataset london son {london.shape}")
london = london.drop(['Unnamed: 0'], axis = 1)

Las dimensiones del dataset london son (51827, 9)


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [5]:
# Verifico la existencia de valores nulos
total_nans = london.isnull().sum().sort_values(ascending=False)
if list(total_nans) != 0:
    print("No existen valores nulos en el dataset")

No existen valores nulos en el dataset


##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [6]:
# Con la siguiente linea de codigo, observamos el tipo de dato, para cada una de nuestras variables
neighbourhood_dtype = london["neighbourhood"].dtype
room_type_dtype = london["room_type"].dtype
print(f"Tipo de dato de 'neighbourhood': {neighbourhood_dtype}")
print(f"Tipo de dato de 'room_type': {room_type_dtype}")

Tipo de dato de 'neighbourhood': object
Tipo de dato de 'room_type': object


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [7]:
# Creo una lista para cada variable
neighbourhood_list = list(london["neighbourhood"])
room_list = list(london["room_type"])
# Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
le = preprocessing.LabelEncoder()
neigh_dummies = le.fit_transform(neighbourhood_list)

Imprimir las dimensiones y primeras lineas de estas dummies

In [8]:
print(f"{type(neigh_dummies)} con {np.shape(neigh_dummies)[0]} filas y 0 columnas")
neigh_dummies[0:10]

<class 'numpy.ndarray'> con 51827 filas y 0 columnas


array([13,  8, 18, 29,  1, 18, 18, 29,  1, 27], dtype=int64)

In [9]:
# Genero las dummies para la variable room_type, dentro de la variable room_dummies.
le1 = preprocessing.LabelEncoder()
room_dummies = le.fit_transform(room_list)

Imprimir las dimensiones y primeras lineas de estas dummies

In [10]:
print(f"{type(room_dummies)} con {np.shape(room_dummies)[0]} filas y 0 columnas")
room_dummies[0:10]

<class 'numpy.ndarray'> con 51827 filas y 0 columnas


array([1, 1, 1, 0, 1, 0, 0, 1, 0, 1], dtype=int64)

Joinear las variables dummies al dataset



In [11]:
# Creo los df para poder concatenarlos con .concat()
neigh_dummies_df = pd.DataFrame([neigh_dummies]).T
room_dummies_df = pd.DataFrame([room_dummies]).T

In [12]:
# Concateno las dos variables dummies al dataset principal
london = pd.concat([london,neigh_dummies_df,room_dummies_df], axis = 1)

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [13]:
y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [14]:
# Divido el dataset entre Train y Test
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=4)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [15]:
# Estandarización de xtrain
scaler = preprocessing.StandardScaler().fit(xtrain)
xtrain_scal = scaler.transform(xtrain)

In [16]:
# Estandarización de xtest
scaler1 = preprocessing.StandardScaler().fit(xtest)
xtest_scal = scaler1.transform(xtest)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [17]:
# Entreno el modelo
reg = LinearRegression().fit(xtrain_scal, ytrain)

In [18]:
# Predigo un array y_pred con valores aprendidos por el modelo mediante x_test
y_pred = reg.predict(xtest_scal)

In [19]:
# Calculo de MSE
mean_squared_error(ytest, y_pred)

1753.243976575432

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [20]:
# Entreno el modelo
svreg = SVR(max_iter=1000)
param_svreg = {'kernel':('linear', 'rbf'), 'C':[0.1, 1, 10 ], 'gamma':[0.01, 0.1, 1]}

In [21]:
reg_svr = GridSearchCV(svreg, param_svreg, cv = 5, refit = True, verbose = 3)

In [22]:
reg_svr.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] C=0.1, gamma=0.01, kernel=linear ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\mativ\anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[CV] ... C=0.1, gamma=0.01, kernel=linear, score=-0.110, total=   1.7s
[CV] C=0.1, gamma=0.01, kernel=linear ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s
C:\Users\mativ\anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.2s remaining:    0.0s


[CV] ... C=0.1, gamma=0.01, kernel=linear, score=-0.103, total=   1.6s
[CV] C=0.1, gamma=0.01, kernel=linear ................................


...

[CV] .......... C=10, gamma=1, kernel=rbf, score=-0.233, total=   2.7s
[CV] C=10, gamma=1, kernel=rbf .......................................


C:\Users\mativ\anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[CV] .......... C=10, gamma=1, kernel=rbf, score=-0.248, total=   2.5s


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  3.1min finished
C:\Users\mativ\anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=1000, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1],
                         'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [23]:
# Mejores hiperparametros
reg_svr.best_params_

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [24]:
# Predigo un array y_pred2 con valores aprendidos por el modelo SVR mediante x_test
y_pred2 = reg_svr.predict(xtest_scal)

In [35]:
# Calculo de MSE
mse1 = mean_squared_error(ytest, y_pred2)
mse1

3579.2297964203613

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [40]:
# Entreno modelo
svreg1 = SVR(max_iter=1000)
param_svreg1 = {'kernel':('linear', 'rbf'), 'C':[0.01, 100, 1000 ], 'gamma':[ 0.0001, 0.001, 10]}
regressor_svr1 = GridSearchCV(svreg1, param_svreg, cv = 5, refit = True, verbose = 5)
regressor_svr1.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] C=0.1, gamma=0.01, kernel=linear ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\mativ\anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] ... C=0.1, gamma=0.01, kernel=linear, score=-0.110, total=   1.7s
[CV] C=0.1, gamma=0.01, kernel=linear ................................


C:\Users\mativ\anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.1s remaining:    0.0s


[CV] ... C=0.1, gamma=0.01, kernel=linear, score=-0.103, total=   1.4s
[CV] C=0.1, gamma=0.01, kernel=linear ................................


...

[CV] .......... C=10, gamma=1, kernel=rbf, score=-0.233, total=   2.4s
[CV] C=10, gamma=1, kernel=rbf .......................................


C:\Users\mativ\anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[CV] .......... C=10, gamma=1, kernel=rbf, score=-0.248, total=   2.9s


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  3.0min finished
C:\Users\mativ\anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=1000, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1],
                         'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=5)

In [41]:
# Mejores hipermparametros
regressor_svr1.best_params_

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [42]:
# Predigo un array y_pred3 con valores aprendidos por el modelo SVR mediante x_test
y_pred3 = regressor_svr1.predict(xtest_scal)

In [43]:
# Nuevo calculo de MSE
mse2 = mean_squared_error(ytest, y_pred3)
mse2

3579.2297964203613

In [44]:
# Comparo desempeño para ambos modelos
if mse1 > mse2:
    print("No se obtuvo una mejora en las predicciones")
else:
    print("Se obtuvo una mejora en las predicciones")

Se obtuvo una mejora en las predicciones
